In [11]:
import os
# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/Google Drive/projects/programming/automation")

from time import sleep

import sistemas
import sei
import functions
import page
from page import *
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

% reload_ext autoreload
% autoreload 2

In [24]:
usr = "rsilva"
pwd = "Savorthemom3nts"

caps = DesiredCapabilities.INTERNETEXPLORER

caps['version'] = '11.0'
caps['requireWindowsFocus'] = True
caps['ignoreProtectedModeSettings'] = True

driver = webdriver.Ie(capabilities=caps)

In [25]:
scpx = sistemas.Scpx(driver)

In [12]:
scpx = sistemas.Scpx(scpx.driver)

In [48]:
import pandas as pd

registros = pd.read_csv("files/53504.0056652018-32.csv", dtype=str)

In [49]:
registros.head()

,CPF,NOME,FISTEL,Validade
0,98008145820,DARCY PEREIRA,80106007688,04/05/2017
1,21053898800,BENEDITO JOSE PEREIRA,80106259067,09/05/2017
2,26556801895,CELSO DOS SANTOS SILVA,80106286200,09/05/2017
3,00691239851,BRAZ NOGUEIRA DA LUZ,80106352199,10/05/2017
4,2758523930,JOEL CARDOSO JUNIOR,80106088904,10/05/2017


In [7]:
def extrai_pares_tabulação(source):
    trs = source.find_all('tr')
    dados = {}
    i = 1
    for tr in trs:
        field = tr.find('td', string=True)
        if field:
            for tag in tr.descendants:
                if tag.name == 'label':
                    label = tag.text
                    key = field.text[:-1]
                    #key += "_" + str(prefix)                        
                    dados[key] = label
    
    return dados

In [50]:
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup

for f in registros.iterrows():
    fistel = f[1].FISTEL
    while len(fistel) < 11:
        fistel = '0' + fistel
    scpx.consulta(fistel,tipo_id='id_fistel')
    source = soup(scpx.driver.page_source, "lxml")
    dados = extrai_pares_tabulação(source)
    
    scpx.wait_for_element_to_click((By.ID, "botaoFlatPróximo")).click()
    val_rf = source.find(id='labelDataValidadeRadioFrequencia')
    
    source = soup(scpx.driver.page_source, "lxml")
    dados = {**dados, **extrai_pares_tabulação(source)}
                                               
    proximo = scpx.wait_for_element_to_click((By.ID, "botaoFlatEstação"))
    proximo.click()
    
    source = soup(scpx.driver.page_source, "lxml")
    dados = {**dados, **extrai_pares_tabulação(source)}
    
    print("Nome: {}".format(f[1]['NOME']))
    print("Validade de Radiofrequência: {}".format(val_rf.text))
    
    banco_de_dados[dados['CNPJ/CPF']] = dados
    
    print("Data de Validade Licença: {}\n".format(dados['Data Validade Licença']))
    
    dados = {}

Nome: DARCY PEREIRA
Validade de Radiofrequência: 05/04/2037
Data de Validade Licença: 05/04/2007

Nome: BENEDITO JOSE PEREIRA
Validade de Radiofrequência: 05/09/2017
Data de Validade Licença: 05/09/2007

Nome: CELSO DOS SANTOS SILVA
Validade de Radiofrequência: 05/09/2017
Data de Validade Licença: 05/09/2007

Nome: BRAZ NOGUEIRA DA LUZ
Validade de Radiofrequência: 05/10/2017
Data de Validade Licença: 05/10/2007

Nome: JOEL CARDOSO JUNIOR
Validade de Radiofrequência: 10/05/2017
Data de Validade Licença: 10/05/2007

Nome: ALBERTO JOSE DE CARVALHO NETO
Validade de Radiofrequência: 11/05/2017
Data de Validade Licença: 11/05/2007

Nome: ALEXANDRE ZAMBOM
Validade de Radiofrequência: 11/05/2017
Data de Validade Licença: 11/05/2007

Nome: ANTONIO CARLOS RIBEIRO BARBOZA
Validade de Radiofrequência: 11/05/2017
Data de Validade Licença: 11/05/2007

Nome: ANTONIO MACHADO NETO
Validade de Radiofrequência: 11/05/2017
Data de Validade Licença: 11/05/2007

Nome: ANTONIO MARIO GABRIEL
Validade de Radio

In [43]:
database = pd.DataFrame()

In [44]:
for v in banco_de_dados.values():
    db = pd.DataFrame([v])
    
    database = database.append(db)

In [45]:
database.head()

,Boleto 2ªVia,Boleto PPDESS,Boleto TFI,CNPJ ou CPF,CNPJ/CPF,Celular,Cep,Data Alteração,Data Emissão Licença,Data Exclusão,...,Situação,Status,Status da Licença,Tipo Usuário,Tipo de Comparação,UF,Validade RNE,Validade Radiofreqüência,Vencidas,Vincendas
0,,801059155640030,,,21451007841,NaN,RUA GABRIEL COURY ATHIE,ANATEL\52967522600 (deleicfaria),,ANATEL\52967522600 (deleicfaria),...,L - Licenciado,L - Licenciado,Licenciamento pendente por PRORROGAÇÃO,3 - Integral,,,,53504.014299/2007-50 (SICAP),,
0,,801060843210003,,,12942851813,NaN,ESTRADA BELVEDERE,ANATEL\13049323892 (luizmoura),ANATEL\13049323892 (luizmoura),,...,L - Licenciado,L - Licenciado,,3 - Integral,,,,53504.007461/2007-83 (SICAP),,
0,,801060528020003,,,13567801805,NaN,RUA GUATEMALA,ANATEL\13049323892 (luizmoura),ANATEL\13049323892 (luizmoura),,...,L - Licenciado,L - Licenciado,,3 - Integral,,,,53504.004642/2007-58 (SICAP),,
0,,801059442380003,,,33650494876,NaN,RUA JOAQUIM NABUCO,ANATEL\25091240802 (rogeriofl),ANATEL\25091240802 (rogeriofl),,...,L - Licenciado,L - Licenciado,,3 - Integral,,,,53504.024685/2006-79 (SICAP),,
0,,801060541710003,,,00171900880,NaN,RUA AMPARO,ANATEL\31888916877 (rsilva),ANATEL\13049323892 (luizmoura),,...,L - Licenciado,L - Licenciado,Licenciamento pendente por ALTERAÇÃO de CNPJ/C...,3 - Integral,,,,53504.004351/2007-60 (SICAP),,


In [23]:
from bs4 import BeautifulSoup as soup
from pprint import pprint

#html = soup(html, 'lxml')

In [15]:
tables = html.find_all('tr')

In [21]:
iterator = iter(tables)

In [49]:
tr = next(iterator)
for child in tr.contents:
    try:
        print(child.attrs)
        print(child.text)
    except:
        pass

{'class': ['SubTituloDireita'], 'width': '15%'}
Cep:
{'class': ['campoesquerda']}
 15104000
{'class': ['SubTituloDireita'], 'width': '15%'}
Logradouro:
{'class': ['campoesquerda'], 'colspan': '5'}
 RUA DOIS


In [5]:
scpx.incluir_serviço(CPF)

In [6]:
scpx.aprovar_movimento(CPF, "B")

In [8]:
scpx.licenciar_estacao(CPF)

In [9]:
scpx.consulta(CPF)

In [17]:
for cpf in CPF[8:]:
    try:
        
        scpx.imprimir_licenca(cpf)
    except UnexpectedAlertPresentException as e:
        pass
    
    input()

In [17]:
datas = ["23052018", "24052018"]
horas = ["1300", "1400"]

horarios = [(dia,hora) for dia in datas for hora in horas][2:]

print(horarios)

sistemas.abrir_agenda_prova(browser, horarios)

[('24052018', '1300'), ('24052018', '1400')]
